In [ ]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D, ReLU, BatchNormalization, Add, ZeroPadding2D, Input
from keras.models import Sequential
import tensorflow as tf
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/CV_dataset/archive.zip -d /content/

## Preprocessing

In [ ]:
IMG_SAVE_PATH = r'/content/Split_smol/train'

In [ ]:
ENCODINGS = {
    'Actinic keratosis': 0,
    'Atopic Dermatitis': 1,
    'Benign keratosis': 2,
    'Dermatofibroma': 3,
    'Melanocytic nevus': 4,
    'Melanoma':5,
    'Squamous cell carcinoma': 6,
    'Tinea Ringworm Candidiasis':7,
    'Vascular lesion': 8
}

NUM_CLASSES = 9


def str_to_Int_mapper(val):
    return ENCODINGS[val]

In [ ]:
import PIL
import cv2

dataset = []
for directory in os.listdir(IMG_SAVE_PATH):
    path = os.path.join(IMG_SAVE_PATH, directory)
    for image in os.listdir(path):
        new_path = os.path.join(path, image)
        try:
            imgpath=PIL.Image.open(new_path)
            imgpath = imgpath.convert('RGB')
            img = np.asarray(imgpath)
            img = cv2.resize(img, (224, 224))
            img = img/255.
            dataset.append([img, directory])
        except FileNotFoundError:
            print('Image file not found. Skipping...')

In [ ]:
data, labels = zip(*dataset)
temp = list(map(str_to_Int_mapper, labels))

In [ ]:
import keras
labels = keras.utils.to_categorical(temp)

In [ ]:
count=0
for i in data:
    if i.shape!=(240, 240, 3):
        count=count+1
print(count)

697


## Skip Connections

In [ ]:
def identity_block(x, filter):
    x_skip = x
    x = Convolution2D(filter, kernel_size = 3, padding="same")(x)
    x = BatchNormalization(axis=3)(x)
    x = ReLU()(x)

    x = Convolution2D(filter, kernel_size = 3, padding="same")(x)
    x = BatchNormalization(axis=3)(x)

    x = Add()([x, x_skip])
    x = ReLU()(x)
    return x


def conv_block(x, filter):
    x_skip = x
    x = Convolution2D(filter, kernel_size = 3, padding="same", strides = 2)(x)
    x = BatchNormalization(axis=3)(x)
    x = ReLU()(x)

    x = Convolution2D(filter, kernel_size = 3, padding="same")(x)
    x = BatchNormalization(axis=3)(x)

    x_skip = Convolution2D(filter, kernel_size=1, strides = 2)(x_skip)

    x = Add()([x, x_skip])
    x = ReLU()(x)
    return x

### ResNet 34

In [ ]:
block_layer_size_34 = [3, 4, 6, 3]
def resnet34(shape):
    x_input = Input(shape)
    # x = ZeroPadding2D((3, 3))(x_input)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding="same")(x_input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=3, strides=2, padding="same")(x)

    filter_size = 64
    for i in range(len(block_layer_size_34)):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layer_size_34[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = conv_block(x, filter_size)
            for j in range(block_layer_size_34[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model


### ResNet 18

In [ ]:
block_layer_size_18 = [2, 2, 2, 2]
def resnet18(shape):
    x_input = Input(shape)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding="same")(x_input)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=3, strides=2, padding="same")(x)

    filter_size = 64
    for i in range(len(block_layer_size_18)):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layer_size_18[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = conv_block(x, filter_size)
            for j in range(block_layer_size_18[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model


In [ ]:
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D, Flatten
model_cnn_res34 = resnet34((224, 224, 3))
model_cnn_res34.compile(
    optimizer=Adam(learning_rate=0.009),         # lr = learning_rate
    loss='categorical_crossentropy',            # Multi-class classification
    metrics=['accuracy'],
)

In [ ]:
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D, Flatten
model_cnn_res18 = resnet18((224, 224, 3))
model_cnn_res18.compile(
    optimizer=Adam(learning_rate=0.003),          # lr = learning_rate
    loss='categorical_crossentropy',              # Multi-class classification
    metrics=['accuracy']
)

In [ ]:
model_cnn_res34.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_36 (Conv2D)          (None, 112, 112, 64)         9472      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_33 (Ba  (None, 112, 112, 64)         256       ['conv2d_36[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_33 (ReLU)             (None, 112, 112, 64)         0         ['batch_normalization_3

In [ ]:
model_cnn_res18.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_72 (Conv2D)          (None, 112, 112, 64)         9472      ['input_3[0][0]']             
                                                                                                  
 batch_normalization_66 (Ba  (None, 112, 112, 64)         256       ['conv2d_72[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_66 (ReLU)             (None, 112, 112, 64)         0         ['batch_normalization_6

In [ ]:
history_cnn_res34 = model_cnn_res34.fit(np.array(data), np.array(labels), epochs = 50, shuffle = True, validation_split = 0.15)

In [ ]:
history_cnn_res18 = model_cnn_res18.fit(np.array(data), np.array(labels), epochs = 50, shuffle = True, validation_split = 0.15)

In [ ]:
# save the model for later use
model_cnn_res34.save("skin_disease_model_ISIC_resnet34.h5")

In [ ]:
# save the model for later use
model_cnn_res18.save("skin_disease_model_ISIC_resnet18.h5")

In [ ]:
IMG_SAVE_PATH_TESTING = r'/content/Split_smol/val'

In [ ]:
import PIL
import cv2
dataset_testing = []
for directory in os.listdir(IMG_SAVE_PATH_TESTING):
    path = os.path.join(IMG_SAVE_PATH_TESTING, directory)
    for image in os.listdir(path):
        new_path = os.path.join(path, image)
        imgpath=PIL.Image.open(new_path)
        imgpath=imgpath.convert('RGB')
        img = np.asarray(imgpath)
        img = cv2.resize(img, (224, 224))
        img=img/255.
        dataset_testing.append([img, directory])

In [ ]:
testing_data, testing_labels = zip(*dataset_testing)
testing_temp = list(map(str_to_Int_mapper, testing_labels))

In [ ]:
import keras
testing_labels = keras.utils.to_categorical(testing_temp)

In [ ]:
model_cnn_res34.evaluate(np.array(testing_data), np.array(testing_labels))

In [ ]:
model_cnn_res18.evaluate(np.array(testing_data), np.array(testing_labels))

## Confusion Matrix

In [ ]:
# Create a confusion matrix
!pip install scikit-plot
from sklearn.metrics import confusion_matrix, classification_report
import scikitplot

# Make predictions
def make_preds(model):
    label_preds = model.predict(np.array(testing_data))

    def one_hot_highest(arr):
        for row in arr:
            max_val = max(row)
            for i in range(len(row)):
                if row[i] == max_val:
                    row[i] = 1
                else:
                    row[i] = 0
        return arr

    arr=one_hot_highest(label_preds) #one hot
    arr2= one_hot_highest(testing_labels)

    labelenc=[]
    for i in range (0, len(arr)):
        flag=0
        for j in range (0,9):
            if arr[i][j]==1:
                labelenc.append(j)
                flag=1
    labelenctest=[]
    for i in range (0, len(arr2)):
        for j in range (0,9):
            if arr2[i][j]==1:
                labelenctest.append(j)

    scikitplot.metrics.plot_confusion_matrix(labelenctest,labelenc)
    report = classification_report(labelenctest,labelenc)
    print(report)

In [ ]:
make_preds(model_cnn_res34)

In [ ]:
make_preds(model_cnn_res18)